In [ ]:
#Kütüphaneler yüklenir.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import folium

df = pd.read_csv("C:\\Users\\evata\\datascience\\earthquake.csv")
df.head()
#Süreye Bağlı Büyüklük (Md)+++

#Yerel (Lokal) Büyüklük (Ml)

#Yüzey Dalgası Büyüklüğü (Ms)+++

#Cisim Dalgası Büyüklüğü (Mb)+++

#Moment Büyüklüğü (Mw)

#deprem şiddeti (xm) 0.0 değerler için hesaplanmadı demek

In [ ]:
df["country"].unique()

In [ ]:
#Türkiye bağlı enlem boylam sınırlandırıldı
df = df[df["country"] == "turkey"]

df=df[df["lat"]>=36]
df=df[df["lat"]<=42]
df=df[df["long"]<=45]
df=df[df["long"]>=26]

In [ ]:
#toplam bos değer
df.isna().sum()

In [ ]:
df.shape


In [ ]:
df.describe()

In [ ]:
#missingno kayıp verileri bulmak için kurulur
import missingno as mno
mno.matrix(df, figsize = (20, 6))

In [ ]:
#mw sutununda cok fazla bos satır bulunduğu için sutun silinir.
df = df.drop(columns= "mw")

In [ ]:
df.tail()

In [ ]:
#son 1697 değer bos olduğu için dikkate alınmadı
df=df[:9949]
mno.matrix(df, figsize = (20, 6))

In [ ]:
df.isnull().sum()

In [ ]:
#area sutununda bos kısımlar için bilinmiyor eklendi.
df = df.fillna("bilinmiyor")

In [ ]:
data = df.copy()

In [ ]:
data['time'] = pd.to_datetime(data['time'])
data['time'] = pd.to_numeric(pd.to_datetime(data['time']))


In [ ]:
# date sütunu gün yıl ay a ayrılır
data[["year", "month", "day"]] = data["date"].str.split(".", expand = True)

In [ ]:
data["year"]= data["year"].astype(int)
data["month"]= data["month"].astype(int)
data["day"]= data["day"].astype(int)

In [ ]:
# şiddetine göre haritalanması 
data.plot(x="long", y="lat", kind="scatter", c="xm", colormap="hot",figsize=(25, 10))

In [ ]:
data.info()

In [ ]:
import geopandas
from folium import GeoJson
from folium import plugins

In [ ]:
lat = np.asarray(data.lat)
lon = np.asarray(data.long)
mag = np.asarray(data.xm)
year = np.asarray(data.year)

mean_lat = data.lat.mean()
mean_lon = data.long.mean()

In [ ]:
location = [lat, lon]

In [ ]:
m= folium.Map(location=[mean_lat, mean_lon], zoom_start=7)
m

In [ ]:
data["lat"]= data["lat"].astype(float)
data["long"]= data["long"].astype(float)
data["xm"]= data["xm"].astype(float)
data["year"]= data["year"].astype(int)

In [ ]:
for idx in range(len(lat)):
    marker = folium.CircleMarker(
        location=[lat[idx],lon[idx]],
        radius=mag[idx]**5/2500,
        color='Red',
        popup=f'lat: {lat[idx]}, long: {lon[idx]} , year: {year[idx]}',
        tooltip=f'Magnitude: {mag[idx]}')
    marker.add_to(m)
# deprem şiddetine göre çap büyüyen haritalandırma

In [ ]:
import fiona
import pandas as pd
import geopandas
import geopandas as gpd

In [ ]:
import geopandas as gpd
gdf = gpd.read_file('C:\\Users\\evata\\datascience\\Turkey_shapefile\\tr_10km.shp')
json = gpd.GeoSeries([gdf.geometry.iloc[0]]).__geo_interface__
# türkiye sınırları için shp dosyası 

In [ ]:
data_map_geo = geopandas.GeoDataFrame(
    data, geometry=geopandas.points_from_xy(data.long, data.lat))

In [ ]:
data_map_geo

In [ ]:
countries = geopandas.read_file("C:\\Users\\evata\datascience\\Turkey_population_with_geopandas-main\\Turkey_population_with_geopandas-main\\countries.geojson")

In [ ]:
countries

In [ ]:
countries.plot()

In [ ]:
turkey = countries[countries["ADMIN"] == "Turkey"]
turkey.plot()

In [ ]:
turkey_gf = geopandas.read_file("C:\\Users\\evata\datascience\\Turkey_population_with_geopandas-main\\Turkey_population_with_geopandas-main\\TUR_adm\\TUR_adm1.shp")

In [ ]:
turkey_geo = turkey_gf.copy()

In [ ]:
turkey_geo = turkey_geo[["NAME_1","geometry"]]

In [ ]:
turkey_geo.rename(columns={"NAME_1":"provience"}, inplace=True)

In [ ]:
turkey_geo.plot(figsize=(15,15),edgecolor="b", facecolor="none")

In [ ]:
turkey_geo.crs #crs EPSG:4326> tipinde

In [ ]:
data_map_geo = gdf.to_crs("EPSG:4326")
# data_map_geo nun da crs EPSG:4326 olarak ayarlanır

In [ ]:
data_map_geo.crs

In [ ]:
data_map_geo

In [ ]:
from shapely.geometry import Polygon, LineString, Point

In [ ]:
data = data.loc[~np.isnan(data["lat"])]
data_merge_turkey = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.long, data.lat))

In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))
turkey_geo.plot(ax=ax, alpha=0.2, color="grey")
data_merge_turkey.plot(column="xm", ax=ax, legend=True)
plt.title("Earthquake")